<a href="https://colab.research.google.com/github/theFishy-1/OCR/blob/OCR/tesseractv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pybind11
!pip install fastwer
!pip install pytesseract
!sudo apt install tesseract-ocr
!pip install pytesseract pillow pdf2image
!apt-get update
!apt-get install -y poppler-utils tesseract-ocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastwer: filename=fastwer-0.1.3-cp311-cp311-linux_x86_64.whl size=866384 sha256=460e07f1bd90d5117e06aa73ae9cfe4ae411ca3d4310b35b487ff1c1b5942959
  Stored in directory: /root/.cache/pip/wheels/4c/53/1e/8d806da8c1ed1de60e371005658af32b92aad9426b37208f1f
Successfully built fastwer
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 23 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 htt

In [2]:
import cv2
import pytesseract
import fastwer
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image, display
from IPython import get_ipython
from PIL import Image
from pdf2image import convert_from_path


In [15]:
folder_path = "./sample_images"
files = sorted([f for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg', '.pdf'))])

df_output = pd.DataFrame(columns=['img_filename', 'ocr_output'])


custom_config = r'--oem 3 --psm 6 -l pol'

for file in files:
    file_path = os.path.join(folder_path, file)


    if file.endswith(('.png', '.jpg', '.jpeg')):
        im = cv2.imread(file_path)
        if im is None:
            print(f"Błąd wczytywania obrazu: {file}")
            continue
        img_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        output = pytesseract.image_to_string(img_rgb, config=custom_config)


    elif file.endswith('.pdf'):
        try:
            images = convert_from_path(file_path)  # Konwersja PDF na obrazy
            output = ""
            for i, img in enumerate(images):
                text = pytesseract.image_to_string(img, config=custom_config)
                output += f"=== Strona {i+1} ===\n{text}\n"
        except Exception as e:
            print(f"Błąd przetwarzania PDF: {file}, {e}")
            continue

    print(f"OCR dla {file}:\n{output}\n{'-'*50}")


    dictionary = {'img_filename': file, 'ocr_output': str(output)}
    df_output = pd.concat([df_output, pd.DataFrame([dictionary])], ignore_index=True)


    output_filename = os.path.join(folder_path, f"{file}.txt")
    with open(output_filename, "w", encoding="utf-8") as text_file:
        text_file.write(output)

OCR dla PNG1.png:
Wrocław, 24.08.2022
Jan Kowalski
ul. Piękna |
50-100 Wrocław
tel. 600 100 200
€mail y kowalski Gmail com
Prezes zarządu
Anna Nowak
Pol. Bud Sj
56-400 Oleśnica
PODANIE
Zwracam uprzejmą prosb
o a © zez r ec 7350WeJ UMOWY O pracę na czas
» z kezliacji dyplomu
budownictwa zywym równicz nadmienić że w doch zaa o ET a
Fak 9" przez mojego przełożonego, kierownika dza: Zaopaz
P zwiatkowskiego Zatrudnienia,
Pedebaadzo wdzięczny z pozytywne rozparzem mojej proby
Z wyrazamu szacunku
Jan Kowalski
Załączniki:
Załącznik nr 1 —
Załączniku 2 opa c dyPoI z mżynira budownctwa

--------------------------------------------------
OCR dla PNG2.png:
RY Warszawa, dn 9 sierpnia 2016 r
MINISTER OBRONY NARODOWEJ
UKaTA wata zatana
A JAWNA PREZES
AISE RUCHU OBYWATELSKIEGO MIŁOŚNIKÓW BRONI
206-00-09 r Pan Andrzej TURCZYN
ul Chłodna 20/109
00-891 Warszawa
Dotyczy: zmany znand dostępu dobrom pono
Sams Rae Rom

"onpowedźi Ba pamo z dni 28 czerwca 2016 r, Sprawie zmiany
zażyczy A ONE ar, r Ty
wiązany

In [16]:
# Create new columns for reference, CER and WER
df_output['ref_text'] = ''
df_output['cer'] = ''
df_output['wer'] = ''

# Replace new lines in output
df_output['ocr_output'] = df_output['ocr_output'].apply(lambda x: x.replace('\n',''))

df_output.loc[df_output['img_filename'] == 'PNG1.png', 'ref_text'] = "Wrocław, 24.08.2022 Jan Kowalski ul. Piękna 1 50-100 Wrocław tel. 600 100 200 e-mail. j.kowalski@gmail.com Prezes zarządu Anna Nowak Przedsiębiorstwo Pol-Bud Sp. z o.o. ul. Ładna 1 56-400 Oleśnica PODANIE Zwracam się z uprzejmą prośbą o przedłużenie dotychczasowej umowy o pracę na czas określony oraz o zwiększenie stawki wynagrodzenia. Swoją prośbę motywuję zwiększeniem moich kwalifikacji tj. uzyskaniem dyplomu inżyniera budownictwa. Chciałbym również nadmienić, że w dotychczasowym okresie zatrudnienia, moja praca była pozytywnie oceniana przez mojego przełożonego, kierownika działu zaopatrzenia p. Marka Kwiatkowskiego. Będę bardzo wdzięczny za pozytywne rozpatrzenie mojej prośby. Z wyrazami szacunku Jan Kowalski Załączniki: Załącznik nr 1 - kserokopia dyplomu inżyniera budownictwa Załącznik nr 2-opinia kierownika działu zaopatrzenia"
df_output.loc[df_output['img_filename'] == 'PNG2.png', 'ref_text'] = "Warszawa, dn. 09 sierpnia 2016 r MINISTER OBRONY NARODOWEJ 2016-08-09 RUCHU OBYWATELSKIEGO MIŁOŚNIKÓW BRONI Pan Andrzej TURCZYN ul. Chłodna 20/109 00-891 Warszawa Dotyczy: zmiany zasad dostępu do broni palnej. W odpowiedzi na pismo z dnia 28 czerwca 2016 r., w sprawie zmiany zasad dostępu do broni palnej, uprzejmie informuję, że w ramach prac związanych z procesem tworzenia Wojsk Obrony Terytorialnej, trwają również prace nad stworzeniem systemu przechowywania i użytkowania broni palnej przez żołnierzy obrony terytorialnej. Aktualnie rozwazane są dwa warianty przechowywania broni palnej Pierwszy z nich przewiduje przechowywanie broni i sprzętu wojskowego w aktualnie istniejących jednostkach wojskowych. W wariancie drugim, w perspektywie dwóch lat, planuje się aby część broni osobistej przechowywać wykorzystując w tym celu wolne miejsca w magazynach broni Policji oraz innych służb mundurowych, na terenie powiatów. Proponowany przez Ruch Obywatelski Miłośników Broni wariant przechowywania broni palnej przez żołnierzy, w miejscu ich zamieszkania jest ciekawymi wartym rozważenia rozwiązaniem, jednak wymaga przeprowadzenia szerokich konsultacji społecznych oraz gruntownych zmian prawnych organizacyjnych w Silach Zbrojnych RP, z tego powodu będzie możliwy do zastosowania w WOT najwcześniej za kilka lat. W przypadku podjęcia decyzji o rozpoczęciu prac legislacyjnych przewidujących praktyczną realizację trzeciego wariantu, stosowny projekt zostanie przesłany w ramach konsultacji do Ruchu Obywatelskiego Miłośników Brani. Przedstawiając powyższe, pragnę podziękować Panu Prezesowi za zainteresowanie omawianymi kwestiami. wz. Bartosz KOWNACKi"
df_output.loc[df_output['img_filename'] == 'PNG3.jpg', 'ref_text'] = "RZECZPOSPOLITA POLSKA MINISTERSTWO FINANSÓW SEKRETARZ STANU Artur Soboń Warszawa, 8 lipca 2022 roku Sprawa: Odpowiedź na pismo - minimalny podatek dochodowy - podmioty lecznicze Znak sprawy: DD6.8223.14.2022 Kontakt: Kancelaria MF tel. +48 22 694 55 55 e-mail: kancelaria@mf.gov.pl Pan Władysław Perchaluk Prezes Związku Szpitali Powiatowych Województwa Śląskiego ul. Żeromskiego 7 41-902 Bytom biuro@szpitalepowiatowe.pl Szanowny Panie Prezesie, w związku z wystąpieniem skierowanym 7 lutego 2022 r.1, zawierającym postulat nowelizacji ustawy CIT2, celem wyłączenia z opodatkowania minimalnym podatkiem dochodowym (minimalny CIT) podmiotów leczniczych udzielających świadczeń opieki zdrowotnej finansowanych ze środków publicznych, proszę przyjąć następujące wyjaśnienia. 1 stycznia 2022 r. wszedł w życie art. 24ca ustawy CIT wprowadzający opodatkowanie minimalnym CIT. Mając na uwadze wpływające do Ministerstwa Finansów zastrzeżenia - podnoszone zarówno przez organizacje przedsiębiorców skupiające poszczególne branże, doradców podatkowych, jak i samych podatników a także uwzględniając sytuację geopolityczną, wynikającą z czynników zewnętrznych, podjęta została decyzja o konieczności przeprowadzenia weryfikacji założeń, które legły u podstaw konstrukcji tego podatku. Założeniami jakie przyświecały wspomnianym analizom były: korekta przepisów budzących wątpliwości interpretacyjne lub prowadzących do rezultatów sprzecznych z ideą regulacji o minimalnym CIT, większe uwzględnienie specyfiki poszczególnych branż, 1 przekazanym Ministerstwu Finansów przy piśmie Biura Prezesa Kancelarii Prezesa Rady Ministrów z 15 lutego 2022 r. znak BPRM.5032.1.21.2022(2)JG 2 Ustawa z 15 lutego 1992 r. o podatku dochodowym od osób prawnych (Dz. U. z 2021 r. poz. 1800, z późn.zm.). Ministerstwo Finansów ul. Świętokrzyska 12 00-916 Warszawa tel.: +48 22 694 55 55 fax: +48 22 694 36 84 e-mail: kancelaria@mf.gov.pl"
df_output.loc[df_output['img_filename'] == 'RĘCZNY1.png', 'ref_text'] = "Warszawa, 27 lutego 2025 r. URZĄD SKARBOWY WARSZAWA ŚRÓDMIEŚCIE UL. NOWOGRODZKA 3/5 00-513 WARSZAWA JAN KOWALSKI UL. KLONOWA 15/3 00-001 WARSZAWA DOTYCZY: INFORMACJA O ZWROCIE NADPŁATY PODATKU DOCHODOWEGO ZA ROK 2024 Szanowny Panie, Urząd Skarbowy Warszawa Śródmieście informuje, że w wyniku rozliczenia rocznego podatku dochodowego od osób fizycznych za rok 2024 stwierdzono nadpłatę w wysokości 1250,00 zł. Zwrot nadpłaty zostanie dokonany na wskazany w zeznaniu podatkowym rachunek bankowy o numerze 12 3456 7890 1234 5678 9012 3456 w terminie do 30 dni od daty doręczenia niniejszego pisma. W przypadku jakichkolwiek pytań lub niezgodności prosimy o kontakt z urzędem skarbowym pod numerem telefonu: 22 123 4567 lub osobiście w siedzibie urzędu. Z poważaniem, Marek Nowak Naczelnik Urzędu Skarbowego Warszawa Śródmieście"
df_output.loc[df_output['img_filename'] == 'RĘCZNY2.png', 'ref_text'] = "NACZELNIK URZĘDU SKARBOWEGO WROCŁAW-FABRYCZNA Wrocław, 26 lutego 2023 roku UNP: SPRAWA: INFORMACJA O HIPOTETYCZNYM PODATKU NALEŻNYM ZNAK SPRAWY: 0224-7010.6624508.2023 KONTAKT: URZĄD SKARBOWY WROCŁAW-FABRYCZNA ALEKSANDRA OSTROWSKIEGO 5 53-238 WROCŁAW INFORMACJA o zwrocie podatku należnego w kwocie 124,00 zł. Szanowny(a) Panie(i), 1 lipca 2022 r. zostały wprowadzone zmiany w przepisach ustawy o podatku dochodowym od osób fizycznych, w tym została zlikwidowana tzw. ulga dla klasy średniej. Nowe przepisy przewidują jednocześnie zabezpieczenie dla podatników, którzy tracą na likwidacji tej ulgi. Wprowadzono możliwość dokonania zwrotu różnicy podatku należnego w przypadku, gdy przy uwzględnieniu ulgi dla klasy średniej obliczony hipotetyczny podatek należny jest niższy niż wskazany w zeznaniu podatkowym za rok 2022."
df_output.loc[df_output['img_filename'] == 'RĘCZNY3.png', 'ref_text'] = "Warszawa, 27 lutego 2025 r. JAN KOWALSKI UL. KLONOWA 15/3 00-001 WARSZAWA TEL. 600 123 456 URZĄD SKARBOWY WARSZAWA-ŚRÓDMIEŚCIE UL. NOWOGRODZKA 3/5 00-513 WARSZAWA DOTYCZY: PROŚBA O ROZŁOŻENIE ZALEGŁOŚCI PODATKOWEJ NA RATY Szanowni Państwo, Zwracam się z uprzejmą prośbą o rozłożenie na raty należności z tytułu podatku dochodowego za rok 2024 w wysokości 5200 zł. Powodem mojej prośby jest trudna sytuacja finansowa spowodowana utratą pracy oraz niespodziewanymi wydatkami związanymi z leczeniem. W związku z powyższym wnoszę o rozłożenie płatności na 6 równych rat po 867 zł każda. Jestem gotów przedstawić dodatkowe dokumenty potwierdzające moją sytuację, jeśli będzie to konieczne. Z poważaniem, JAN KOWALSKI"

for index, row in df_output.iterrows():
  filename = row['img_filename']
  ref = row['ref_text']
  output = row['ocr_output']
  cer = fastwer.score_sent(output, ref, char_level=True)
  wer = fastwer.score_sent(output, ref, char_level=False)
  df_output.loc[df_output['img_filename'] == filename, 'cer'] = round(cer,2) # Round value to 2 decimal places
  df_output.loc[df_output['img_filename'] == filename, 'wer'] = round(wer,2)

df_output

,img_filename,ocr_output,ref_text,cer,wer
0,PNG1.png,"Wrocław, 24.08.2022Jan Kowalskiul. Piękna |50-...","Wrocław, 24.08.2022 Jan Kowalski ul. Piękna 1 ...",48.99,84.55
1,PNG2.png,"RY Warszawa, dn 9 sierpnia 2016 rMINISTER OBRO...","Warszawa, dn. 09 sierpnia 2016 r MINISTER OBRO...",41.61,79.07
2,PNG3.jpg,SN Pzd Ż» RZECZPOSPOLITA POLSKABS ZSS MINISTER...,RZECZPOSPOLITA POLSKA MINISTERSTWO FINANSÓW SE...,6.42,37.3
3,RĘCZNY1.png,"WARSZAWA, 2% LUTEGO 2036R.URZĄD SKARBOWY MKRSZ...","Warszawa, 27 lutego 2025 r. URZĄD SKARBOWY WAR...",68.27,98.32
4,RĘCZNY2.png,( NACZELNIK URZĘDUSKARBOLEGO MROLEAA-FABRYCZNĄ...,NACZELNIK URZĘDU SKARBOWEGO WROCŁAW-FABRYCZNA ...,66.63,104.67
5,RĘCZNY3.png,"WARSZAWA, 27 LUTEGOOJAN KOWALSKI KORE G.UL. KL...","Warszawa, 27 lutego 2025 r. JAN KOWALSKI UL. K...",62.95,92.23
